In [2]:
import pandas as pd
import pickle

In [ ]:
import pickle

# FOR MARCH MADNESS
boxscores_final.to_csv('march_madness/boxscores_final_lookup.csv',index=False)


with open('march_madness/march_madness_home_model.pkl','wb') as home_mdl_pkl:
    pickle.dump(home_model,home_mdl_pkl)

with open('march_madness/march_madness_away_model.pkl','wb') as away_mdl_pkl:
    pickle.dump(away_model,away_mdl_pkl)

In [16]:
boxscores_final = pd.read_csv('boxscores_final_lookup.csv')

with open('march_madness_home_model.pkl', 'rb') as h:
    home_model = pickle.load(h)

with open('march_madness_away_model.pkl', 'rb') as a:
    away_model = pickle.load(a)

teams = pd.read_csv('teams.csv')
home_features=['fgm_home',
 '2pm_home',
 '2pa_home',
 '3pm_home',
 'ftm_home',
 'fta_home',
 'oreb_home',
 'dreb_home',
 'reb_home',
 'ast_home',
 'stl_home',
 'blk_home',
 'to_home',
 'pf_home',
 'awayTeam_losses',
 'fgm_away',
 'fga_away',
 '2pm_away',
 '2pa_away',
 '3pm_away',
 '3pa_away',
 'ftm_away',
 'fta_away',
 'oreb_away',
 'reb_away',
 'ast_away',
 'stl_away',
 'blk_away',
 'to_away',
 'pf_away']

away_features= ['fga_home',
 '2pa_home',
 '3pm_home',
 '3pa_home',
 'ftm_home',
 'fta_home',
 'oreb_home',
 'dreb_home',
 'reb_home',
 'stl_home',
 'blk_home',
 'to_home',
 'pf_home',
 'homeTeam_losses',
 'fgm_away',
 'fga_away',
 '2pm_away',
 '2pa_away',
 '3pm_away',
 '3pa_away',
 'ftm_away',
 'oreb_away',
 'dreb_away',
 'reb_away',
 'ast_away',
 'stl_away',
 'blk_away',
 'pf_away']

feature_cols=['fgm_home', 'fga_home', '2pm_home', '2pa_home', '3pm_home', '3pa_home',
       'ftm_home', 'fta_home', 'oreb_home', 'dreb_home', 'reb_home',
       'ast_home', 'stl_home', 'blk_home', 'to_home', 'pf_home',
       'is_conference', 'is_neutral', 'homeTeam_wins', 'homeTeam_losses',
       'awayTeam_wins', 'awayTeam_losses', 'fgm_away', 'fga_away', '2pm_away',
       '2pa_away', '3pm_away', '3pa_away', 'ftm_away', 'fta_away', 'oreb_away',
       'dreb_away', 'reb_away', 'ast_away', 'stl_away', 'blk_away', 'to_away',
       'pf_away']

/Users/mabbate/michael-abbate/cbb-predictor/venv/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.2.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/mabbate/michael-abbate/cbb-predictor/venv/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RidgeClassifier from version 1.2.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [86]:
def predictOutcome(game_day, home_team_name, away_team_name,is_neutral=1, is_conference=0):
    game_date_str = game_day
    game_date = pd.to_datetime(game_date_str)
    boxscores_final['game_day'] = pd.to_datetime(boxscores_final['game_day'])
    home_most_recent_bs = pd.DataFrame(boxscores_final[(boxscores_final['team']==home_team_name) & (boxscores_final['game_day']<game_date)].iloc[-1,:]).transpose()
    away_most_recent_bs = pd.DataFrame(boxscores_final[(boxscores_final['team']==away_team_name) & (boxscores_final['game_day']<game_date)].iloc[-1,:]).transpose()

    home_team_cols_needed = {}
    for f in feature_cols:
        if '_home' in f:
            home_team_cols_needed[f.replace('_home','')]=f
    home_most_recent_bs=home_most_recent_bs.rename(columns=home_team_cols_needed)
    home_most_recent_bs=home_most_recent_bs.drop(['awayTeam_wins','awayTeam_losses'],axis=1)

    away_team_cols_needed = {}
    for f in feature_cols:
        if '_away' in f:
            away_team_cols_needed[f.replace('_away','')]=f
    away_most_recent_bs=away_most_recent_bs.rename(columns=away_team_cols_needed)
    away_most_recent_bs=away_most_recent_bs.drop(['homeTeam_wins','homeTeam_losses'],axis=1)

    home_most_recent_bs['join_helper']=1
    away_most_recent_bs['join_helper']=1

    validate_df = pd.merge(home_most_recent_bs, away_most_recent_bs,on='join_helper')
    validate_df['is_neutral']=is_neutral
    validate_df['is_conference']=is_conference

    home_pred = home_model.predict(validate_df[home_features]) 
    away_pred = away_model.predict(validate_df[away_features])
    # print(home_team_name, home_pred[0])
    # print(away_team_name, away_pred[0])
    winning_team = home_team_name if home_pred[0]>away_pred[0] else away_team_name
    return {'game_day':game_date_str,"winning_team": winning_team, home_team_name: str(home_pred[0]), away_team_name: str(away_pred[0])}


In [75]:
lookup_home = 'Tennessee'
lookup_away = 'Texas A'
print(teams.loc[teams['team'].str.contains(lookup_home,case=True),'team'])
print(teams.loc[teams['team'].str.contains(lookup_away,case=True),'team'])


14        Tennessee Tech Golden Eagles
15                Tennessee Volunteers
183      Middle Tennessee Blue Raiders
279    East Tennessee State Buccaneers
359             Tennessee State Tigers
Name: team, dtype: object
22               Texas A&M-Commerce Lions
23                       Texas A&M Aggies
332    Texas A&M-Corpus Christi Islanders
Name: team, dtype: object


In [87]:

rightside = [
    {'game_day':'2024-03-21','home': 'Houston Cougars', 'away': 'Longwood Lancers'},
    {'game_day':'2024-03-21','home': 'Nebraska Cornhuskers', 'away': 'Texas A&M Aggies'},
    {'game_day':'2024-03-21','home': 'Wisconsin Badgers', 'away': 'James Madison Dukes'},
    {'game_day':'2024-03-21','home': 'Duke Blue Devils', 'away': 'Vermont Catamounts'},
    {'game_day':'2024-03-21','home': 'Texas Tech Red Raiders', 'away': 'NC State Wolfpack'},
    {'game_day':'2024-03-21','home': 'Kentucky Wildcats', 'away': 'Oakland Golden Grizzlies'},
    {'game_day':'2024-03-21','home': 'Florida Gators', 'away': 'Colorado Buffaloes'},
    {'game_day':'2024-03-21','home': 'Marquette Golden Eagles', 'away': 'Western Kentucky Hilltoppers'},
    {'game_day':'2024-03-21','home': 'Purdue Boilermakers', 'away': 'Grambling Tigers'},
    {'game_day':'2024-03-21','home': 'Utah State Aggies', 'away': 'TCU Horned Frogs'},
    {'game_day':'2024-03-21','home': 'Gonzaga Bulldogs', 'away': 'McNeese Cowboys'},
    {'game_day':'2024-03-21','home': 'Kansas Jayhawks', 'away': 'Samford Bulldogs'},
    {'game_day':'2024-03-21','home': 'South Carolina Gamecocks', 'away': 'Oregon Ducks'},
    {'game_day':'2024-03-21','home': 'Creighton Bluejays', 'away': 'Akron Zips'},
    {'game_day':'2024-03-21','home': 'Texas Longhorns', 'away': 'Colorado State Rams'},
    {'game_day':'2024-03-21','home': 'Tennessee Volunteers', 'away': 'Saint Peter\'s Peacocks'},
]

leftside = [
    {'game_day':'2024-03-21','home':'UConn Huskies', 'away': 'Stetson Hatters'},
    {'game_day':'2024-03-21','home': 'Florida Atlantic Owls', 'away': 'Northwestern Wildcats'},
    {'game_day':'2024-03-21','home': 'San Diego State Aztecs', 'away': 'UAB Blazers'},
    {'game_day':'2024-03-21','home': 'Auburn Tigers', 'away': 'Yale Bulldogs'},
    {'game_day':'2024-03-21','home': 'BYU Cougars', 'away': 'Duquesne Dukes'},
    {'game_day':'2024-03-21','home': 'Illinois Fighting Illini', 'away': 'Morehead State Eagles'},
    {'game_day':'2024-03-21','home': 'Washington State Cougars', 'away': 'Drake Bulldogs'},
    {'game_day':'2024-03-21','home': 'Iowa State Cyclones', 'away': 'South Dakota State Jackrabbits'},
    {'game_day':'2024-03-21','home': 'North Carolina Tar Heels', 'away': 'Wagner Seahawks'},
    {'game_day':'2024-03-21','home': 'Mississippi State Bulldogs', 'away': 'Michigan State Spartans'},
    {'game_day':'2024-03-21','home': 'Saint Mary\'s Gaels', 'away': 'Grand Canyon Lopes'},
    {'game_day':'2024-03-21','home': 'Alabama Crimson Tide', 'away': 'Charleston Southern Buccaneers'},
    {'game_day':'2024-03-21','home': 'Clemson Tigers', 'away': 'New Mexico Lobos'},
    {'game_day':'2024-03-21','home': 'Baylor Bears', 'away': 'Colgate Raiders'},
    {'game_day':'2024-03-21','home': 'Dayton Flyers', 'away': 'Nevada Wolf Pack'},
    {'game_day':'2024-03-21','home': 'Arizona Wildcats', 'away': 'Long Beach State Beach'}, 
]


In [89]:
games = []
for row in rightside:
    try:
        outcome=predictOutcome(row['game_day'],row['home'],row['away'])
    except:
        print('MISSED:', row)
    games.append(outcome)
for row in leftside:
    outcome=predictOutcome(row['game_day'],row['home'],row['away'])
    games.append(outcome)

In [90]:
import json
g = json.dumps(games)

In [92]:
for g in games:
    print(g['winning_team'])

Houston Cougars
Nebraska Cornhuskers
James Madison Dukes
Duke Blue Devils
Texas Tech Red Raiders
Kentucky Wildcats
Florida Gators
Marquette Golden Eagles
Purdue Boilermakers
Utah State Aggies
Gonzaga Bulldogs
Samford Bulldogs
South Carolina Gamecocks
Creighton Bluejays
Texas Longhorns
Tennessee Volunteers
UConn Huskies
Florida Atlantic Owls
UAB Blazers
Auburn Tigers
BYU Cougars
Illinois Fighting Illini
Washington State Cougars
Iowa State Cyclones
North Carolina Tar Heels
Mississippi State Bulldogs
Saint Mary's Gaels
Alabama Crimson Tide
Clemson Tigers
Baylor Bears
Dayton Flyers
Arizona Wildcats


In [93]:
round32 = [
    {'game_day':'2024-03-21','home':'UConn Huskies', 'away': 'Florida Atlantic Owls'},
    {'game_day':'2024-03-21','home':'Auburn Tigers', 'away': 'UAB Blazers'},
    {'game_day':'2024-03-21','home':'Illinois Fighting Illini', 'away': 'BYU Cougars'},
    {'game_day':'2024-03-21','home':'Iowa State Cyclones', 'away': 'Washington State Cougars'},
    {'game_day':'2024-03-21','home':'North Carolina Tar Heels', 'away': 'Mississippi State Bulldogs'},
    {'game_day':'2024-03-21','home':'Alabama Crimson Tide', 'away': 'Saint Mary\'s Gaels'},
    {'game_day':'2024-03-21','home':'Baylor Bears', 'away': 'Clemson Tigers'},
    {'game_day':'2024-03-21','home':'Arizona Wildcats', 'away': 'Dayton Flyers'},
    {'game_day':'2024-03-21','home':'Houston Cougars', 'away': 'Nebraska Cornhuskers'},
    {'game_day':'2024-03-21','home':'Duke Blue Devils', 'away': 'James Madison Dukes'},
    {'game_day':'2024-03-21','home':'Kentucky Wildcats', 'away': 'Texas Tech Red Raiders'},
    {'game_day':'2024-03-21','home':'Marquette Golden Eagles', 'away': 'Florida Gators'},
    {'game_day':'2024-03-21','home':'Purdue Boilermakers', 'away': 'Utah State Aggies'},
    {'game_day':'2024-03-21','home':'Gonzaga Bulldogs', 'away': 'Samford Bulldogs'},
    {'game_day':'2024-03-21','home':'Creighton Bluejays', 'away': 'South Carolina Gamecocks'},
    {'game_day':'2024-03-21','home':'Tennessee Volunteers', 'away': 'Texas Longhorns'},
]

In [101]:
games32 = []
for row in round32:
    try:
        outcome=predictOutcome(row['game_day'],row['home'],row['away'])
    except:
        print('MISSED:', row)
    games32.append(outcome)

In [102]:
for g in games32:
    print(g['winning_team'])

UConn Huskies
Auburn Tigers
Illinois Fighting Illini
Washington State Cougars
North Carolina Tar Heels
Alabama Crimson Tide
Baylor Bears
Arizona Wildcats
Houston Cougars
Duke Blue Devils
Kentucky Wildcats
Marquette Golden Eagles
Purdue Boilermakers
Gonzaga Bulldogs
Creighton Bluejays
Tennessee Volunteers


In [100]:
round16 = [
    {'game_day':'2024-03-21','home':'UConn Huskies', 'away': 'Auburn Tigers'},
    {'game_day':'2024-03-21','home':'Illinois Fighting Illini', 'away': 'Washington State Cougars'},
    {'game_day':'2024-03-21','home':'North Carolina Tar Heels', 'away': 'Alabama Crimson Tide'},
    {'game_day':'2024-03-21','home':'Arizona Wildcats', 'away': 'Baylor Bears'},
    {'game_day':'2024-03-21','home':'Houston Cougars', 'away': 'Duke Blue Devils'},
    {'game_day':'2024-03-21','home':'Marquette Golden Eagles', 'away': 'Kentucky Wildcats'},
    {'game_day':'2024-03-21','home':'Purdue Boilermakers', 'away': 'Gonzaga Bulldogs'},
    {'game_day':'2024-03-21','home':'Tennessee Volunteers', 'away': 'Creighton Bluejays'},
]

In [103]:
games16 = []
for row in round16:
    try:
        outcome=predictOutcome(row['game_day'],row['home'],row['away'])
    except:
        print('MISSED:', row)
    games16.append(outcome)


In [105]:
for g in games16:
    print(g['winning_team'])

UConn Huskies
Illinois Fighting Illini
North Carolina Tar Heels
Arizona Wildcats
Houston Cougars
Marquette Golden Eagles
Purdue Boilermakers
Tennessee Volunteers


In [106]:
round8 = [
    {'game_day':'2024-03-21','home':'UConn Huskies', 'away': 'Illinois Fighting Illini'},
    {'game_day':'2024-03-21','home':'North Carolina Tar Heels', 'away': 'Arizona Wildcats'},
    {'game_day':'2024-03-21','home':'Houston Cougars', 'away': 'Marquette Golden Eagles'},
    {'game_day':'2024-03-21','home':'Purdue Boilermakers', 'away': 'Tennessee Volunteers'},
]

In [110]:
games8 = []
for row in round8:
    try:
        outcome=predictOutcome(row['game_day'],row['home'],row['away'])
    except:
        print('MISSED:', row)
    games8.append(outcome)


In [111]:
for g in games8:
    print(g['winning_team'])

UConn Huskies
North Carolina Tar Heels
Marquette Golden Eagles
Purdue Boilermakers


In [113]:
final4 = [
    {'game_day':'2024-03-21','home':'UConn Huskies', 'away': 'North Carolina Tar Heels'},
    {'game_day':'2024-03-21','home':'Purdue Boilermakers', 'away': 'Marquette Golden Eagles'},
]

In [114]:
games4 = []
for row in final4:
    try:
        outcome=predictOutcome(row['game_day'],row['home'],row['away'])
    except:
        print('MISSED:', row)
    games4.append(outcome)


In [115]:
games4

[{'game_day': '2024-03-21',
  'winning_team': 'UConn Huskies',
  'UConn Huskies': '87',
  'North Carolina Tar Heels': '76'},
 {'game_day': '2024-03-21',
  'winning_team': 'Purdue Boilermakers',
  'Purdue Boilermakers': '87',
  'Marquette Golden Eagles': '76'}]

In [116]:
predictOutcome('2024-03-21','UConn Huskies','Purdue Boilermakers')

{'game_day': '2024-03-21',
 'winning_team': 'UConn Huskies',
 'UConn Huskies': '87',
 'Purdue Boilermakers': '79'}